# Asignación y Desenmascarado

Saquemos las máscaras a los tweets

In [1]:
import pandas as pd

df_titulares = pd.read_csv("../data/titulos_filtrados.csv")

df_titulares = df_titulares.rename(columns={"Unnamed: 0": "id"}).set_index("id")

df_titulares

,medio,fecha,titulo
id,,,
23,la izquierda diario,2019-09-21 00:00:00,Barrio Aeroclub: la Izquierda propone urbaniza...
24,la izquierda diario,2019-09-21 00:00:00,Elecciones municipales: el #FITUnidad busca fo...
31,la izquierda diario,2019-09-21 00:00:00,[Videos] Mirá todos los spots del Frente de Iz...
65,la nación,2019-09-22 00:00:00,Tipo de cambio: por qué nos preocupa el dólar ...
75,la izquierda diario,2019-09-22 00:00:00,Brutalidad de la Policía de Urtubey en el reci...
...,...,...,...
22608,prensa obrera,2019-10-27 23:05:46,Declaración del Partido Obrero-Frente de Izqui...
22611,la nación,2019-10-27 23:16:00,"Resultados elecciones 2019: Alberto Fernández,..."
22612,la nación,2019-10-27 23:21:00,Elecciones 2019. La llamada de Macri a Alberto...


In [2]:
import pandas as pd
import glob
import numpy as np
import re



paths = glob.glob("../data/etiquetados_final/*.xlsx")

label_cols = ['TARGET A', 'TARGET B', 'TARGET C', 'TARGET D',
       'TARGET E', 'TARGET F']

dfs = []
annotators = []

for path in paths:
    print(path)
    name = os.path.basename(path).split(".")[0]
    
    df = pd.read_excel(path, engine='openpyxl', index_col=0)
    df = df.iloc[3:]
    df = df.loc[
        df.index.notna()
    ]

    #df.index = df.index.astype(int)
    annotators.append(name)

    df.rename(columns={df.columns[0]: "id", "TITULO": "titulo_mask", " ": "etiquetado", "Etiquetado?": "etiquetado"}, inplace=True)

    # Por algún motivo perdimos este titular...
    ids = [x for x in df.index if x not in {39}]
    
    df.loc[ids, "titulo"] = df_titulares.loc[ids, "titulo"]
    df.loc[ids, "medio"] = df_titulares.loc[ids, "medio"]

    for label_col in label_cols:
        df[label_col] = df[label_col].apply(lambda x: x.strip() if isinstance(x, str) else x)
    # KJJJJ qué desastre esto
    df.loc[39, "titulo"] = "Entre el milagro de Macri y la grieta del peronismo"
    df.loc[39, "medio"] = "la nación"
    df["anotador"] = name
    df.reset_index(inplace=True)
    dfs.append(df)
    
annotators = sorted(annotators)
annotator_df = pd.concat(dfs).set_index(["index", "anotador"])

../data/etiquetados_final/e2.xlsx
../data/etiquetados_final/e3.xlsx
../data/etiquetados_final/et1.xlsx


## Problemas de etiquetado

Si tiene algo que no sea POS, NEG, NEU => lo "blanqueo"

In [3]:
import numpy as np


for label in label_cols:
    problematic = (annotator_df[label].notna() & ~annotator_df[label].apply(lambda x: x in {"POS", "NEG", "NEU"}))
    annotator_df.loc[problematic, label] = np.nan

In [4]:
annotator_df.loc[22381, "titulo"] = 'Por error, y en plena veda, TN dio por ganador a Alberto Fernández'
annotator_df.loc[18149, "titulo_mask"] = '[TARGET D]: "Chile despertó, fuerza al pueblo chileno"'

annotator_df.loc[10205, "titulo_mask"] = "Grabois intenta marcarle la cancha [TARGET E]: “Si no cumple las demandas saldremos a protestar”"
annotator_df.loc[10205, "TARGET E"] = "NEG"


annotator_df.loc[11154, "titulo_mask"] = "El mercado apuesta a la moderación de [TARGET E]: suben bonos y baja el riesgo país"
annotator_df.loc[11154, "TARGET E"] = "POS"


annotator_df.loc[13076, "titulo_mask"] = "El [TARGET E] sumó su provincia número 11 en 2019 y es favorito en otras cuatro"
annotator_df.loc[13076, "TARGET E"] = "POS"


annotator_df.loc[12266, "TARGET C"] = np.nan

annotator_df.loc[13900, "TARGET E"] = "NEU"


annotator_df.loc[14378, "TARGET E"] = np.nan
annotator_df.loc[(14378, "e3"), "TARGET C"] = "POS"

annotator_df.loc[15166, "TARGET C"] = np.nan

annotator_df.loc[(17468, "e3"), "TARGET D"] = "POS"

annotator_df.loc[18149, "TARGET A"] = np.nan

annotator_df.loc[(18198, "e2"), "TARGET C"] = "NEU"

#OK
annotator_df.loc[18448, "TARGET C"] = np.nan

annotator_df.loc[(18474, "e3"), "TARGET E"] = "POS"
annotator_df.loc[(18474, "e3"), "TARGET C"] = np.nan

annotator_df.loc[(18999, "e3"), "TARGET E"] = np.nan

annotator_df.loc[(14286, "e3"), "TARGET D"] = np.nan
annotator_df.loc[(14286, "e3"), "TARGET B"] = "POS"

annotator_df.loc[(19022, "et1"), "TARGET E"] = "NEU"
annotator_df.loc[(19022, "et1"), "TARGET C"] = np.nan

annotator_df.loc[(22104, "et1"), "TARGET C"] = "POS"

annotator_df.loc[(22224, "e3"), "TARGET E"] = "NEU"

annotator_df.loc[(22456, "e3"), "TARGET D"] = "POS"

annotator_df.loc[(22583, "e2"), "TARGET C"] = "NEG"


# Chequeados
annotator_df.loc[2219, "titulo_mask"] = '[TARGET C] retuvo Mendoza con un contundente triunfo sobre PJ'
annotator_df.loc[2219, "TARGET B"] = np.nan
annotator_df.loc[2219, "TARGET C"] = "POS"



annotator_df.loc[(2010, "e3"), "TARGET E"] = "NEU"

annotator_df.loc[2049, "TARGET C"] = np.nan


annotator_df.loc[5677, "TARGET E"] = np.nan
annotator_df.loc[5677, "TARGET D"] = "POS"

annotator_df.loc[6379, "TARGET E"] = "NEU"

annotator_df.loc[6566, "TARGET C"] = np.nan

annotator_df.loc[(6566, "et1"), "TARGET E"] = "NEG"
annotator_df.loc[6379, "TARGET E"] = np.nan

annotator_df.loc[6571, "TARGET E"] = np.nan
annotator_df.loc[(6571, "et1"), "TARGET C"] = "NEU"

annotator_df.loc[7438, "titulo_mask"] = "El mapa de gobernadores para el nuevo presidente, con más peso del [TARGET E]"
annotator_df.loc[7438, "TARGET E"] = "NEU"


annotator_df.loc[8387, "TARGET C"] = np.nan
annotator_df.loc[8387, "TARGET E"] = "POS"


annotator_df.loc[8392, "TARGET E"] = np.nan
annotator_df.loc[8392, "TARGET C"] = "NEG"

# Hasta acá OK

## Asignación

Primero, veamos cómo asignarlo

In [5]:
ids = dfs[0]["index"]

rejected = []

accepted_ids = []

for _, row in dfs[0].iterrows():
    idx = row["index"]
    labels = annotator_df.loc[idx]

    # Ver que haya 3 anotaciones
    assert len(labels) == 3
    # Ver si tienen el titulo_mask igual
    assert len(labels["titulo_mask"].value_counts()) == 1
    annotated = labels[label_cols].notna()

    """
    Chequeamos si la anotación difirió
    """
    if not ( all(annotated.iloc[0] == annotated.iloc[1]) and all(annotated.iloc[1] == annotated.iloc[2])):
        """
        print("="*80 + "\n" * 2)
        print(idx, row["titulo"])
        print(annotated.sum(axis=1))
        print(annotated)
        """
        rejected.append(idx)
    else:
        accepted_ids.append(idx)

print("Titulares aceptados: ", len(accepted_ids))
print("Titulares descartados: ", len(rejected))

Titulares aceptados:  2234
Titulares descartados:  19


Ok, quedémonos con accepted_ids

Para cada titular, hagamos lo siguiente:

- Votación mayoritaria (elegimos al que tenga 2 votos)
- Si no ocurre...mmm, tiremos el titular mejor

In [6]:
skip = {1601, 2163, 3296, 3537, 4253, 4411, 6562, 6603, 8935, 11154, 11478, 11229, 11688, 11982, 12082, 12749, 21820, 21883, 22222, 22312}

important_rejected = [x for x in rejected if x not in skip]
sorted(important_rejected)

[]

In [7]:
import re
from tqdm.auto import tqdm

def get_targets(instance):
    mascarado = re.sub(r"\s+", " ", instance["titulo_mask"]).strip()
    titulo = instance["titulo"]

    i = 0
    j = 0

    found_targets = {}

    while i < len(titulo):
        # print("="*80)
        # print(titulo[i:])
        # print(mascarado[j:])
        # print(found_targets)
        if titulo[i] == mascarado[j]:
            i += 1
            j += 1
        else:
            "son distintos, tiene que ser un target"
            #import ipdb; ipdb.set_trace()
            assert mascarado[j] == "["
            
            next_idx = mascarado[j:].index("]") + j 
            
            target = mascarado[j+1:next_idx]

            j = next_idx + 1

            target_name = ""
            if j >= len(mascarado)-1:
                """
                Listo, me quedo con todo
                """
                target_name = titulo[i:]
                i = len(titulo)
            else:
                """
                Todavía queda algo
                """
                
                pattern = mascarado[j:j+2]
                next = titulo[i:].index(pattern)
                target_name = titulo[i:i+next]
                i = i + next 
            
            if target in found_targets:
                found_targets[target].append(target_name)
            else:
                found_targets[target] = [target_name]
    return found_targets



In [8]:
import unidecode

problematic = []

dataset = []

for idx in tqdm(accepted_ids):
    annotated_columns = [col for col, annotated in annotator_df.loc[idx, label_cols].notna().all().items() if annotated]

    # Agarro un etiquetador, total da igual
    row = annotator_df.loc[(idx, "et1")]

    ret = {k:row[k] for k in ["titulo", "titulo_mask"]}
    ret["id"] = idx
    mascara = re.sub(r"\s+", " ", ret["titulo_mask"]).strip()
    mascara = re.sub(r"(\])\s+([^\w\s“])", r"\1\2", mascara)

    ret["titulo_mask"] = mascara 
    ret["medio"] = unidecode.unidecode(row["medio"])
    ret["targets"] = {}
    """
    Asignamos
    """
    for col in annotated_columns:
        counts = annotator_df.loc[idx, col].value_counts()

        """
        Si no hay ningún máximo (es decir, si los votos son POS, NEG, NEU) => descartamos instancia
        """
        if counts.max() == 1:
            problematic.append(idx)
        else:
            label = counts.index[counts.argmax()]
            ret["targets"][col] = label


    if ret["targets"]:
        try:
            ret["mapping"] = get_targets(ret)
        except:
            pass
        dataset.append(ret)
print(f"Hay {len(problematic)} casos problemáticos de etiquetas (anotaron POS, NEG, NEU)")

100%|██████████| 2234/2234 [00:09<00:00, 241.97it/s]

Hay 113 casos problemáticos de etiquetas (anotaron POS, NEG, NEU)


In [9]:
pd.options.display.max_colwidth = 200

annotator_df.loc[set(problematic)]["titulo_mask"].apply(lambda x: x.count("TARGET")).value_counts() / 3

1    79.0
2    24.0
3     4.0
0     1.0
Name: titulo_mask, dtype: float64

In [10]:
set_ids = []

def set_mapping(idx, mapping, keep=False):
    if keep:
        set_ids.append(idx)
    instance = next(x for x in dataset if x["id"] == idx)

    for k, v in mapping.items():
        if not isinstance(v, list):
            mapping[k] = [v]
    instance["mapping"] = mapping


set_mapping(21210, {"TARGET C": ["Juntos por el Cambio", "Macri"]})
set_mapping(22428, {"TARGET C": ["Juntos por el Cambio"]})
set_mapping(4806, {"TARGET D": ["Del Caño", "Del Plá", "FIT"]})

set_mapping(263, {"TARGET D": "Frente de Izquierda"})
set_mapping(639, {"TARGET E": "Alberto"})
set_mapping(690, {"TARGET D": ["Nicolás del Caño", "Romina del Pla", "FIT U"]})
set_mapping(827, {"TARGET E": ["Alberto Fernández", "Cristina Kirchner"]})
set_mapping(1042, {"TARGET C": ["Pichetto", "Macri"]})
set_mapping(1094, {"TARGET E": ["Alberto Fernández", "Cristina Kirchner"]})
set_mapping(1167, {"TARGET C": ["Juntos por el Cambio", "Macri"] })
set_mapping(1264, {"TARGET E": ["Alberto Fernández", "Cristina Kirchner"]})
set_mapping(1282, {"TARGET C": ["Miguel Ángel Pichetto", "Macri"], "TARGET E": ["Fernández", "Cristina"]})
set_mapping(1287, {"TARGET B": ["Roberto Lavagna", "Juan Manuel Urtubey"]})
set_mapping(1291, {"TARGET C": "Gobierno", "TARGET E": ["Alberto F.", "Cristina Kirchner"]}, keep=False)
set_mapping(2060, {"TARGET C": ["Gobierno", "Mauricio Macri"]}, keep=False)
set_mapping(2057, {"TARGET C": "Mauricio Macri", "TARGET E": "Alberto Fernández"})
set_mapping(2158, {"TARGET C": ["Gobierno", "Macri"]}, keep=False)
set_mapping(2423, {"TARGET C": "Gobierno", "TARGET E": "Alberto Fernández"}, keep=False)
set_mapping(2434, {"TARGET C": ["Miguel Ángel Pichetto", "Gobierno"]})
set_mapping(2725, {"TARGET C": ["Miguel Ángel Pichetto", "Gobierno"]})
set_mapping(2835, {"TARGET C": ["Pichetto", "Gobierno"]})

set_mapping(3446, {"TARGET C": ["Gobierno", "Macri"]})
set_mapping(4388, {"TARGET C": ["Gobierno", "Macri"]})

set_mapping(4600, {"TARGET E": "Alberto"})
set_mapping(4783, {"TARGET A": "Espert", "TARGET D": "Nico del Caño"})
set_mapping(4885, {"TARGET B": "Roberto Lavagna", "TARGET C": "Gobierno"})
set_mapping(4936, {"TARGET B": ["Roberto Lavagna", "Consenso Federal"], "TARGET C": "Gobierno"})
set_mapping(5061, {"TARGET C": "Gobierno", "TARGET E": "Alberto Fernández"})
set_mapping(5338, {"TARGET C": "Mauricio Macri", "TARGET E": "peronismo"})
set_mapping(5453, {"TARGET E": "Alberto"})
set_mapping(6301, {"TARGET C": "Juntos por el Cambio"})
set_mapping(5070, {"TARGET A": "José Luis Espert", "TARGET D": "Nicolás del Caño"})
set_mapping(5804, {"TARGET E": ["Alberto Fernández", "gobierno k"], "TARGET C": ["este gobierno"]})
set_mapping(6014, {"TARGET B": "Roberto Lavagna", "TARGET C": "Gobierno"})
set_mapping(6605, {"TARGET C": "Gobierno", "TARGET E": "Alberto Fernández"})
set_mapping(7429, {"TARGET C": "Gobierno", "TARGET E": "Alberto Fernández"})
set_mapping(7725, {"TARGET C": "gobierno", "TARGET E": "Alberto Fernández"})
set_mapping(8131, {"TARGET C": "gobierno", "TARGET E": "Alberto Fernández"})
set_mapping(10204, {"TARGET E": "PJ"})
set_mapping(10513, {"TARGET B": "Roberto Lavagna", "TARGET C": "Mauricio Macri", "TARGET E": "Alberto Fernández"})
set_mapping(10748, {"TARGET C": "gobierno", "TARGET E": "Alberto Fernández"})
set_mapping(11474, {"TARGET C": "gobierno", "TARGET E": "Alberto Fernández"})
set_mapping(11528, {"TARGET C": ["Gobierno", "Macri"]})

set_mapping(12462, {"TARGET C": "Gobierno", "TARGET E": "Cristina"})
set_mapping(13068, {"TARGET D": "Nicolás del Caño"})
set_mapping(14842, {"TARGET C": ["Juntos por el Cambio", "Macri"]})
set_mapping(14855, {"TARGET C": ["Gobierno", "Macri"]})

set_mapping(14150, {"TARGET C": "Macri", "TARGET E": "Alberto F."})
set_mapping(15856, {"TARGET B": "Roberto Lavagna", "TARGET C": "Gobierno"})
set_mapping(16255, {"TARGET C": ["Gobierno", "Macri"]})
set_mapping(16134, {"TARGET B": "Lavagna", "TARGET C": "Gobierno"})
set_mapping(16424, {"TARGET C": "Mauricio Macri", "TARGET E": "Alberto Fernández"})
set_mapping(17935, {"TARGET C": "Gobierno", "TARGET E": "CFK"})
set_mapping(18449, {"TARGET C": "Mauricio Macri", "TARGET E": "Alberto Fernández"})
set_mapping(18231, {"TARGET D": "la izquierda"})
set_mapping(18467, {"TARGET A": "José Luis Espert", "TARGET C": "Mauricio Macri", "TARGET E": "kirchnerismo"})
set_mapping(19447, {"TARGET C": "Gobierno"})
set_mapping(20433, {"TARGET C": "Mauricio Macri"})
set_mapping(21233, {"TARGET C": "Mauricio Macri", "TARGET E": "Alberto Fernández"})
set_mapping(21483, {"TARGET C": "Gobierno", "TARGET E": "peronismo"})
set_mapping(22381, {"TARGET E": "Alberto Fernández"})
#set_mapping(2434, {"TARGET C": ["Miguel Ángel Pichetto", "Gobierno"]})
delete_ids = [2219]

dataset = [x for x in dataset if x["id"] not in delete_ids]

[x for x in dataset if "mapping" not in x or x["id"] in set_ids]

[]

In [11]:
ignore = {639}

for instance in dataset:
    if instance["id"] in ignore:
        continue
    if instance["titulo_mask"].count("TARGET") != sum([1 if isinstance(v, str) else len(v) for k, v in instance["mapping"].items()]):
        print("="*80)
        print(instance["id"])
        print(instance["titulo"])
        print(instance["titulo_mask"], instance["targets"], instance["mapping"])

10204
El PJ quiere revisar tarifas y asociarlas con el costo de producción
El PJ quiere revisar tarifas y asociarlas con el costo de producción {'TARGET E': 'NEU'} {'TARGET E': ['PJ']}


In [12]:
target_mapping = {
        'TARGET A': 'ESPERT',
        'TARGET B': 'LAVAGNA',
        'TARGET C':'JXC',
        'TARGET D': 'FIT',
        'TARGET E': 'FDT',
        'TARGET F': 'NOS'
}

def clean_instance(instance):
    ret = {k:instance[k] for k in ["titulo", "titulo_mask", "id", "medio"]}

    ret["targets"] = []
    
    for target_name, label in instance["targets"].items():
        names = instance["mapping"][target_name]

        if isinstance(names, str):
            names = [names]

        for name in names:
            ret["targets"].append({
                "nombre": name,
                "partido": target_mapping[target_name],
                "etiqueta": label
            })
    return ret

clean_dataset = []
for instance in dataset:
    clean_dataset.append(clean_instance(instance))
#clean_dataset = [clean_instance(instance) for instance in dataset]
    
#clean_dataset = [clean_instance(instance) for instance in dataset]
print("Cantidad de titulares: ", len(clean_dataset))
print("Cantidad de targets: ", sum([len(l["targets"])for l in clean_dataset]))



Cantidad de titulares:  1976
Cantidad de targets:  2439


## Sanity check

Veamos que todos los targets estén adentro del título

In [13]:
ignore_ids = {690, 2158}
for instance in clean_dataset:
    if instance["id"] in ignore_ids:
        continue
    for target in instance["targets"]:
        if target["nombre"].lower() not in instance["titulo"].lower():
            print(instance)
            print(k)

In [15]:
for instance in clean_dataset:

    if instance["id"] in {639, 2057, 4783, 5070, 5338, 10204}:
        continue
    
    if instance["titulo_mask"].count("TARGET") != len(instance["targets"]):
        print(instance)

{'titulo': 'El dilema de Macri: ¿cruzar el Rubicón y emitir o dejarle el balurdo a Alberto F.?', 'titulo_mask': 'El dilema de [TARGET C]: ¿cruzar el Rubicón y emitir o dejarle el balurdo a [TARGET E].?', 'id': 10295, 'medio': 'ambito financiero', 'targets': [{'nombre': 'Macri', 'partido': 'JXC', 'etiqueta': 'NEG'}]}
{'titulo': 'Mauricio Macri y Roberto Lavagna rechazan la suba de Bienes Personales que planteó Alberto Fernández', 'titulo_mask': '[TARGET C] y [TARGET B] rechazan la suba de Bienes Personales que planteó [TARGET E]', 'id': 10513, 'medio': 'la nacion', 'targets': [{'nombre': 'Alberto Fernández', 'partido': 'FDT', 'etiqueta': 'NEG'}]}
{'titulo': 'Cristina juega al silencio pero influye en la campaña y los proyectos de gobierno', 'titulo_mask': '[TARGET E] juega al silencio pero influye en la campaña y los proyectos de GOBIERNO ([TARGET C])', 'id': 12462, 'medio': 'clarin', 'targets': [{'nombre': 'Gobierno', 'partido': 'JXC', 'etiqueta': 'NEU'}]}
{'titulo': '¿Es verdad que el

Nada grave

In [16]:
clean_dataset = [{k:v for k, v in instance.items() if k != "titulo_mask"} for instance in clean_dataset]

len(clean_dataset)

1976

In [17]:
import json

with open("../data/dataset.json", "w+") as f:
    json.dump(clean_dataset, f, indent=4)